In [4]:
import pathlib as pt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms

In [5]:
data_dir = pt.Path(r'T:\current\Projects\Deere\CPSI\2021\Digital Acre\Furrow_Vision\Residue\Dataset')

In [6]:

transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()
                                ])# TODO: compose transforms here
dataset = datasets.ImageFolder(data_dir, transform=transform) # TODO: create the ImageFolder
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)


In [7]:
# for images, labels in dataloader:
#     pass

# Get one batch
images, labels = next(iter(dataloader))

In [8]:
import torch
from torch import nn
import torchvision.models as models

In [18]:
# Deciding which device to used
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print('Using {} device'.format(device))

learning_rate = 1e-06
batch_size = 64
pretrain = False
model_structure = 'Resnet34'
debug = True

Using cuda device


In [10]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1, 1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)

    def forward(self, x):
        return torch.cat([self.mp(x), self.ap(x)], 1)


In [19]:
class YieldModel(nn.Module):
    global pretrain, model_structure, batch_size

    def __init__(self):
        super().__init__()
        layers = list(models.resnet50(pretrained=pretrain).children())
        layers[0] = nn.Conv2d(3, batch_size, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.YieldModel = nn.Sequential(*layers)

    def forward(self, x):
        return self.YieldModel(x).squeeze(-1)
        # could add 116*torch.sigmoid


In [20]:
model = YieldModel().to(device)
print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

C:\ProgramData\Anaconda3\envs\Notebook\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\Notebook\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


YieldModel(
  (YieldModel): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2

In [21]:
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()

    # Returns the function that will be called inside the train loop
    return train_step

In [22]:
# Creates the train_step function for our model, loss function and optimizer
train_step = make_train_step(model, loss_fn, optimizer)
losses = []
n_epochs = 1000

In [23]:
for epoch in range(n_epochs):
    print(f"Running for {epoch} :")
    for batch, (X, y) in enumerate(dataloader):
        x_train_tensor = X.to(device)
        y_train_tensor = y.to(device)

        if debug:
            print("Debug mode: tensor shape")
            print(x_train_tensor.shape, y_train_tensor.shape)
            # break

        # Performs one train step and returns the corresponding loss
        loss = train_step(x_train_tensor, y_train_tensor)
        losses.append(loss)

    if debug:
        print("Debug mode: Breaking Epoch loop")
        break
# Checks model's parameters
if not debug:
    print(model.state_dict())

Running for 0 :
Debug mode: tensor shape
torch.Size([32, 3, 224, 224]) torch.Size([32])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (65536x1 and 2048x1000)